In [2]:
# print version / should be 1.6.3
println(versioninfo())

# load centralized paths dictionary
using YAML
PATHS_FILE = "../../../PATHS.yml"
PATHS = YAML.load_file(PATHS_FILE);

"""
    load_path(keys::Vector{String}):String

Return the absolute path for a requested item within the nested PATHS dictionary.
"""
function loadpath(keys::Vector{String}):String
    # recusively assemble paths from keys
    requested_path = foldl((x, y) -> getindex(x, y), keys; init=PATHS)
    # rephrase abs path from relative paths to be platform independent.
    realpath(joinpath(splitdir(realpath(PATHS_FILE))[1], requested_path))
end;

Julia Version 1.6.3
Commit ae8452a9e0 (2021-09-23 17:34 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU E5-2630 v3 @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
Environment:
  __LMOD_REF_COUNT_JULIA_DEPOT_PATH = /home/biotoml/.julia:1;/sw/comp/julia/1.6.3/rackham/lib/glob_pkg:1
  JULIA_DEPOT_PATH = /home/biotoml/lsm/src/.julia/v1.6_depot:/home/biotoml/.julia:/sw/comp/julia/1.6.3/rackham/lib/glob_pkg
  __LMOD_REF_COUNT_JULIA_LOAD_PATH = @:1;@v#.#:1;@stdlib:1;/sw/comp/julia/1.6.3/rackham/lib/glob_pkg/environments/v1.6:1
  JULIA_PROJECT = /crex/proj/snic2022-23-321/private/thomas/ext_src/01_julia_land_v01/Land/Project.toml
  JULIA_ROOT = /sw/comp/julia/1.6.3/rackham
  JULIA_LOAD_PATH = @:@v#.#:@stdlib:/sw/comp/julia/1.6.3/rackham/lib/glob_pkg/environments/v1.6
nothing


In [3]:
# we load the project path as it is used by CliMA Land v01 papers
using Pkg
PROJECT_PATH = loadpath(["EXTERNAL", "01_CLIMA_LAND_V01", "ROOT"])
Pkg.activate(PROJECT_PATH)

  Activating environment at `/crex/proj/snic2022-23-321/private/thomas/ext_src/01_julia_land_v01/Land/Project.toml`


In [10]:
using LazyArtifacts

using CanopyLayers: BLUE, EVI, FourBandsFittingHybrid, NDVI, NIR, NIRv, NIRvES, RED, SIF_740, SIF_WL, SIF_fluxes!, canopy_fluxes!, canopy_geometry!, canopy_matrices!, fit_soil_mat!, short_wave!, soil_albedos
using DataFrames: DataFrame
using GriddingMachine: lat_ind, lon_ind
using Photosynthesis: AbstractPhotoModelParaSet, AirLayer, leaf_ETR!
using PkgUtility: numerical∫, read_csv, save_csv!
using PlantHydraulics: soil_p_25_swc, temperature_effects!
using SoilPlantAirContinuum: SPACMono, initialize_spac_canopy!, update_Cab!, update_LAI!, zenith_angle
using Statistics: mean
using StomataModels: BetaGLinearPsoil, CanopyLayer, ESMMedlyn, OSMWang, OSMSperry, EmpiricalStomatalModel, GswDrive, gas_exchange!, gsw_control!, stomatal_conductance, update_leaf_TP!, β_factor
using UnPack: @unpack
using WaterPhysics: saturation_vapor_pressure

In [5]:
using PlantHydraulics: create_grass

In [6]:
methods(create_grass)

# 1 method for generic function "create_grass":
[1] create_grass(z_root::FT, z_canopy::FT, soil_bounds::Vector{FT}, air_bounds::Vector{FT}) where FT<:AbstractFloat in PlantHydraulics at /home/biotoml/.julia/packages/PlantHydraulics/Aztny/src/initialize/plant.jl:20

In [4]:
EXAMPLE = Dict(
    # general
    "latitude" => - 105.4,
    "longitude" => 54.6,
    "date" => "2015-02-05T05:00:00",
    "surface_area" => 1.789e+03,
    "elevation" => 396.4,
    
    # winds
    "u10" => 2.523,
    "v10" => -0.9888,
    
    # temperatures
    "d2m" => 255.0,
    "t2m" => 257.6,
    "stl1" => 271.1,
    "stl2" => 271.7,
    "stl3" => 273.3,
    "stl4" => 275.4,
    "skt" => 256.9,
    
    # moisture
    "evavt" => 1.37e-06,
    
    # pressure
    "sp" => 9.467e+04,
    
    # waves
    "fal" => 0.3225,
    "sshf" => 1.372e+06,
    "soil_color" => 19.64,
    
    # plant specifications
    "lai_hv" => 1.422,
    "lai_lv" => 0.0,
    "lai" => 0.8437,
    "biomass" => 0.03891,
    "canopy_height" => 9.5,
    "chlorophyll" => 14.0,
    "clumping_index" => 0.5567,
    "vcmax_rubisco" => 65.34,
    "tree_density" => 6.494e+04,
    
    # soil hydraulics
    "src" => 0.0002756,
    "soil_wcr1" => 0.03256,
    "soil_wcr2" => 0.0327,
    "soil_wcr3" => 0.04254,
    "soil_wcr4" => 0.02991,
    "soil_vga1" => 468.5,
    "soil_vga2" => 541.8,
    "soil_vga3" => 296.1,
    "soil_vga4" => 591.6,
    "soil_vgn1" => 1.142,
    "soil_vgn2" => 1.145,
    "soil_vgn3" => 1.179,
    "soil_vgn4" => 1.154,
    "soil_ksat1" => 202.0,
    "soil_ksat2" => 82.86,
    "soil_ksat3" => 63.88,
    "soil_ksat4" => 60.24,
    "swvl1" => 0.2048,
    "swvl2" => 0.1967,
    "swvl3" => 0.1708,
    "swvl4" => 0.224,
);

In [1]:
# function to create a SPAC node
function create_spac(
        lat::FT,  # core
        lon::FT,  # core
        # elevation::FT,  # can be computed from lat+lon
        
        
        _soil_bnds::AbstractVector{FT} = FT[0, -0.1, -0.35, -1, -3],  # 4 layers of soil with given boundaries (in m) as for JULES - can use ERA5 data
        _z_c::FT = 5.0,  # canopy height should be used from langHighresolutionCanopyHeight2022 or GriddingMachine
        _z_r::FT = -2,  # we set a default 2 m max root depth. Closest reference is fanHydrologicRegulationPlant2017 and https://wci.earth2observe.eu/thredds/catalog/usc/root-depth/catalog.html
        n_airlayers::Int8 = 20,  # not more than 127!
        
        # following components can be taken from GriddingMachine / ERA5
        wind_v10::FT,
        wind_u10::FT,
        wind_d::FT,
        wind_zs::Vector{FT},
        winds::Vector{FT},
        mswc::Vector{FT},
        swc::Vector{FT},
        p_soil::Vector{FT},
        h_soil::FT,
        lai::FT,  # leaf area index from 
        
        # photomodel C3 or C4
        #  can be C3VJPModel, C3CytochromeModel, C4VJPModel
        plant_hs::AbstractPlantOrganism{FT} = create_grass(...)  # vegetation_type - GrassLikeOrganism, TreeLikeOrganism, PalmLikeOrganism (init via create_*)
        
        # Stomatal models - we can try different models - optimization approaches were great in trials
        stomata_model::AbstractStomatalModel{FT} = ESMMedlyn{FT}(),
        
        
        ) where {FT<:AbstractFloat}
    _Δz::FT    = _z_c / n_airlayers;
    _air_bnds  = collect(FT, 0:_Δz:_z_c+2*_Δz);
    _elev::FT  = 100;

    # we compute vector length of wind given u and v component
    wind_z0 = sqrt(wind_v10^2 + wind_u10^2)
    
    # create a SPACMono struct
    _node = SPACMono{FT}(soil_bounds = _soil_bnds, air_bounds = _air_bnds, z_canopy = _z_c, z_root = _z_r, latitude = lat, longitude = lon, elevation = _elev, stomata_model = ESMMedlyn{FT}());

    # initialize the canopy RT model
    initialize_spac_canopy!(_node);

    return _node

end;




LoadError: syntax: invalid identifier name "..."

In [6]:
# VARS
# lai
# chlorophyll
# swvl1 - swvl4
# msdrswrf
# msdwswrf
# DAY
# VPD???
# co2
# wind_z0
# t2m
# skt

# first create the SPAC
create_spac()

# to insert the LAI into all different structures, we use a given function
update_LAI!(node, lai) 

# to insert the chlorophyll and carotinoids
for _leaf in node.leaves_rt
        _leaf.Car = chlorophyll / 7; 
end;
update_Cab!(node, chlorophyll);

# we do gpp prediction if radiation is given
if _e_dir + _e_dif > 10
    # update soil water matrices per layer
    _svc_1 = plant_hs.roots[1].sh;
    _svc_2 = plant_hs.roots[2].sh
    _svc_3 = plant_hs.roots[3].sh
    _svc_4 = plant_hs.roots[4].sh
    _swc_1 = max(_svc_1.Θr+FT(0.001), FT(swvl1));
    _swc_2 = max(_svc_2.Θr+FT(0.001), FT(swvl2));
    _swc_3 = max(_svc_3.Θr+FT(0.001), FT(swvl3));
    _swc_4 = max(_svc_4.Θr+FT(0.001), FT(swvl4));

    _p_1   = soil_p_25_swc(_svc_1, _swc_1);
    _p_2   = soil_p_25_swc(_svc_2, _swc_2);
    _p_3   = soil_p_25_swc(_svc_3, _swc_3);
    _p_4   = soil_p_25_swc(_svc_4, _swc_4);
    plant_hs.roots[1].p_ups = _p_1;
    plant_hs.roots[2].p_ups = _p_2;
    plant_hs.roots[3].p_ups = _p_3;
    plant_hs.roots[4].p_ups = _p_4;

    # update soil albedo
    if hyper
        fit_soil_mat!(node.soil_opt, node.wl_set, _swc_1, _method);
    else
        # TODO make this a method of fit_soil_mat!
        _ρ_PAR,_ρ_NIR = soil_albedos(node.soil_opt.color, _swc_1, true);
        node.soil_opt.ρ_SW .= _ρ_NIR;
        node.soil_opt.ρ_SW[node.wl_set.iPAR] .= _ρ_PAR;
        node.soil_opt.ρ_SW_SIF .= node.soil_opt.ρ_SW[node.wl_set.iWLF];
        node.soil_opt.ε_SW .= 1 .- node.soil_opt.ρ_SW;
    end;

    # update PAR related information
    _e_dir = msdrswrf;
    _e_dif = msdwswrf-msdrswrf;
    in_rad.E_direct  .= _in_rad_bak.E_direct  .* _e_dir ./ _in_dir;
    in_rad.E_diffuse .= _in_rad_bak.E_diffuse .* _e_dif ./ _in_dif;
    angles.sza = min(88, zenith_angle(latitude, FT(DAY)));
    canopy_geometry!(canopy_rt, angles, can_opt, rt_con);
    canopy_matrices!(leaves_rt, can_opt);
    short_wave!(canopy_rt, can_opt, can_rad, in_rad, soil_opt, rt_con);
    canopy_fluxes!(canopy_rt, can_opt, can_rad, in_rad, soil_opt, leaves_rt, wl_set, rt_con);

    # calculate leaf level flux per canopy layer
    for _i_can in 1:n_canopy
        _iPS = plant_ps[_i_can];
        _iRT = n_canopy + 1 - _i_can;

        # calculate the fraction of sunlit and shaded leaves
        _f_view = (can_opt.Ps[_iRT] + can_opt.Ps[_iRT+1]) / 2;
        for iLF in 1:_nSL
            _iPS.APAR[iLF] = can_rad.absPAR_sunCab[(_iRT-1)*_nSL+iLF] * FT(1e6);
            _iPS.LAIx[iLF] = _f_view * f_SL[iLF];
        end;
        _iPS.APAR[end] = can_rad.absPAR_shadeCab[_iRT] * FT(1e6);
        _iPS.LAIx[end] = 1 - _f_view;
    end;
    
    # calculate leaf level flux per canopy layer
    for _i_can in 1:n_canopy
        _iEN = envirs[_i_can];
        _iHS = plant_hs.leaves[_i_can];
        _iPS = plant_ps[_i_can];

        # update environmental conditions
        _iEN.t_air = df.t2m[_i];
        _iEN.p_atm = df.sp[_i];
        _iEN.p_a   = _iEN.p_atm * co2 * 1e-6;
        _iEN.p_O₂  = _iEN.p_atm * 0.209;
        _iEN.p_sat = saturation_vapor_pressure(_iEN.t_air);
        _iEN.vpd   = VPD;
        _iEN.p_H₂O = _iEN.p_sat - _iEN.vpd;
        _iEN.RH    = _iEN.p_H₂O / _iEN.p_sat;
        _iEN.wind  = wind_z0;

        # prescribe leaf temperature
        _iPS.T = max(t2m, skt);
        update_leaf_TP!(photo_set, _iPS, _iHS, _iEN);
        temperature_effects!(_iHS, _iPS.T);

        # iterate for 30 times to find steady state solution
        if _e_dir + _e_dif > 10
            for iter in 1:30
                # beta factor
                _pl = _iHS.p_dos;
                _β1 = β_factor(_iHS, _svc_1, _beta_g, _pl, _p_1, _swc_1);
                _β2 = β_factor(_iHS, _svc_2, _beta_g, _pl, _p_2, _swc_2);
                _β3 = β_factor(_iHS, _svc_3, _beta_g, _pl, _p_3, _swc_3);
                _β4 = β_factor(_iHS, _svc_4, _beta_g, _pl, _p_4, _swc_4);
                _βm = mean([_β1, _β2, _β3, _β4]);

                # calculate the photosynthetic rates
                gas_exchange!(photo_set, _iPS, _iEN, GswDrive());
                # TODO add soil water content corrected beta function
                update_gsw!(_iPS, stomata_model, photo_set, _iEN, FT(120); β=_βm);
                gsw_control!(photo_set, _iPS, _iEN);
            end;
        else
            # set g_sw and APAR to 0, and then gsw_control!
            _iPS.APAR .= 0;
            _iPS.g_sw .= 0;
            gsw_control!(photo_set, _iPS, _iEN);
        end;

    # update the flow rates
        _f_H₂O  += numerical∫(_iPS.g_lw, _iPS.LAIx) * (_iPS.p_sat - _iEN.p_H₂O) / _iEN.p_atm * _iPS.LA;
        _f_CO₂  += numerical∫(_iPS.An, _iPS.LAIx) * _iPS.LA;
        _f_GPP  += numerical∫(_iPS.Ag, _iPS.LAIx) * _iPS.LA;
        _f_APAR += numerical∫(_iPS.APAR, _iPS.LAIx) * FT(1e-6) * canopy_rt.LAI / canopy_rt.nLayer;
    end;

    # save the total flux
    F_H2O = _f_H₂O / ga;
    F_CO2 = _f_CO₂ / ga;
    F_GPP = _f_GPP / ga;
    
end;

32767

2.695752367782323

In [1]:
!which julia

LoadError: syntax: extra token "julia" after end of expression